In [1]:
%%capture
!pip install --upgrade google-api-python-client
!pip install google-auth
!pip install oauth2client

You need to create credentials before you can use the scripts below ( see https://cloud.google.com/docs/authentication/getting-started)

In [2]:
import argparse
import googleapiclient
from googleapiclient import discovery
from six.moves import input
from pprint import pprint
import time
import os
path_to_key = '/Users/pgreisen/Projects/Enevolv/keys_gcp/ngsdataanalysis-08329a276003.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_key

In [3]:
try:
    from oauth2client.client import GoogleCredentials  # pylint: disable=g-import-not-at-top
    import google.auth.exceptions  # pylint: disable=g-import-not-at-top
    from googleapiclient import discovery  # pylint: disable=g-import-not-at-top
    from googleapiclient import errors  # pylint: disable=g-import-not-at-top
except ImportError:
    print("Warning, google-api-python-client or google-auth not installed, can't connect to GCP instances.")

In [4]:
try:
    credentials = GoogleCredentials.get_application_default()
except google.auth.exceptions.DefaultCredentialsError:
    print("No credentials set")

In [5]:
# initiate the resource manager to monitor projects
resourcemanager = discovery.build('cloudresourcemanager', 'v1', credentials=credentials)

In [11]:
# get projects from gcp
projects = [project['projectId']
                    for project in resourcemanager.projects().list().execute().get('projects', [])]

In [31]:
from pprint import pprint

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()

service = discovery.build('compute', 'v1', credentials=credentials)

# Project ID for this request.
project = project_  # TODO: Update placeholder value.

# The name of the zone for this request.
zone = 'us-west2-b'  # TODO: Update placeholder value.

request = service.instances().list(project=project, zone=zone)
while request is not None:
    response = request.execute()

    for instance in response['items']:
        # TODO: Change code below to process each `instance` resource:
        pprint(instance)

    request = service.instances().list_next(previous_request=request, previous_response=response)

KeyError: 'items'

In [27]:
from pprint import pprint

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()

service = discovery.build('compute', 'v1', credentials=credentials)

# Project ID for this request.
project = project_ # 'my-project'  # TODO: Update placeholder value.

request = service.zones().list(project=project)
while request is not None:
    response = request.execute()

    for zone in response['items']:
        # TODO: Change code below to process each `zone` resource:
        pprint(zone)

    request = service.zones().list_next(previous_request=request, previous_response=response)

{'availableCpuPlatforms': ['Intel Skylake', 'Intel Broadwell', 'Intel Haswell'],
 'creationTimestamp': '1969-12-31T16:00:00.000-08:00',
 'description': 'us-east1-b',
 'id': '2231',
 'kind': 'compute#zone',
 'name': 'us-east1-b',
 'region': 'https://www.googleapis.com/compute/v1/projects/ngsdataanalysis/regions/us-east1',
 'selfLink': 'https://www.googleapis.com/compute/v1/projects/ngsdataanalysis/zones/us-east1-b',
 'status': 'UP'}
{'availableCpuPlatforms': ['Intel Skylake', 'Intel Broadwell', 'Intel Haswell'],
 'creationTimestamp': '1969-12-31T16:00:00.000-08:00',
 'description': 'us-east1-c',
 'id': '2233',
 'kind': 'compute#zone',
 'name': 'us-east1-c',
 'region': 'https://www.googleapis.com/compute/v1/projects/ngsdataanalysis/regions/us-east1',
 'selfLink': 'https://www.googleapis.com/compute/v1/projects/ngsdataanalysis/zones/us-east1-c',
 'status': 'UP'}
{'availableCpuPlatforms': ['Intel Skylake', 'Intel Broadwell', 'Intel Haswell'],
 'creationTimestamp': '1969-12-31T16:00:00.000-

In [ ]:
from google.cloud import dns
client = dns.Client(project=prj)
zones = client.list_zones()  # API request
[zone.name for zone in zones]

In [ ]:
GCP_REGION_LIST = ['us-east1', 'asia-east1', 'asia-northeast1','asia-southeast1', 'europe-west1', 'us-central1', 'us-west1']

In [ ]:
credentials = GoogleCredentials.get_application_default()
compute = discovery.build('compute', 'v1', credentials=credentials)
#print(compute.addresses())
#

In [ ]:
result = compute.instances().list(project=project, zone=zone).execute()
#instances = compute.list_instances(compute, project, zone)
#print('Instances in project %s and zone %s:' % (project, zone))
#for instance in instances:
#    print(' - ' + instance['name'])

In [ ]:
#print(compute.addresses().get(address=instance.name, project=instance.project, region=instance.region).execute()['address'])
#try:
#    print(compute.addresses().get(address=instance.name, project=instance.project, region=instance.region).execute()['address'])
#except:
#    print("Done")

result = compute.instances().list(project='ngsdataanalysis',zone=GCP_REGION_LIST[0]).execute()
print(result['items'])

In [ ]:
def list_instances(compute, project, zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items']

In [ ]:
compute = googleapiclient.discovery.build('compute', 'v1')

In [ ]:
def list_instances(compute, project, zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items']

In [ ]:
import argparse
import googleapiclient
from googleapiclient import discovery
from six.moves import input
import time
import os

def main(project, bucket, zone, instance_name, wait=True):
    compute = googleapiclient.discovery.build('compute', 'v1')
	
    print('Creating instance.')

    operation = create_instance(compute, project, zone, instance_name, bucket)
    wait_for_operation(compute, project, zone, operation['name'])

    instances = list_instances(compute, project, zone)

    print('Instances in project %s and zone %s:' % (project, zone))
    for instance in instances:
        print(' - ' + instance['name'])

    print("""
Instance created.
It will take a minute or two for the instance to complete work.
Check this URL: http://storage.googleapis.com/{}/output.png
Once the image is uploaded press enter to delete the instance.
""".format(bucket))

    if wait:
        input()

    print('Deleting instance.')

    operation = delete_instance(compute, project, zone, instance_name)
    wait_for_operation(compute, project, zone, operation['name'])

def list_instances(compute, project, zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items']


def create_instance(compute, project, zone, name, bucket):
    # Get the latest Debian Jessie image.
    image_response = compute.images().getFromFamily(
        project='debian-cloud', family='debian-8').execute()
    source_disk_image = image_response['selfLink']

    # Configure the machine
    machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
    startup_script = open(
        os.path.join(
            os.path.dirname(__file__), 'startup-script.sh'), 'r').read()
    image_url = "http://storage.googleapis.com/gce-demo-input/photo.jpg"
    image_caption = "Ready for dessert?"

    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],

        # Allow the instance to access cloud storage and logging.
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }],

        # Metadata is readable from the instance and allows you to
        # pass configuration from deployment scripts to instances.
        'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script',
                'value': startup_script
            }, {
                'key': 'url',
                'value': image_url
            }, {
                'key': 'text',
                'value': image_caption
            }, {
                'key': 'bucket',
                'value': bucket
            }]
        }
    }

    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()

def wait_for_operation(compute, project, zone, operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)  

def delete_instance(compute, project, zone, name):
    return compute.instances().delete(
        project=project,
        zone=zone,
        instance=name).execute()

if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument('project_id', help='Your Google Cloud project ID.')
    parser.add_argument(
        'bucket_name', help='Your Google Cloud Storage bucket name.')
    parser.add_argument(
        '--zone',
        default='us-central1-c',
        help='Compute Engine zone to deploy to.')
    parser.add_argument(
        '--name', default='cloud-demo-instance', help='New instance name.')

    args = parser.parse_args()

    main(args.project_id, args.bucket_name, args.zone, args.name)